In [1]:
import pandas as pd
import numpy as np

### Indonesia

In [74]:
articles = pd.read_csv("../../data/processed/query_results/content_quality/indonesia/CQ_all_articles.csv")
recs     = pd.read_csv("../../data/processed/query_results/topic_lists/indonesia/rec_qids_iwls.csv", sep=',', encoding = 'utf-8') 
pv_df    = pd.read_csv("../../data/raw/articles/2019/query_results/content_quality/per_wiki_full/Indonesia/articles_w_july_incountry_pageviews.csv", sep=',', encoding = 'utf-8')

articles_labels_df = pd.read_csv("../../data/processed/query_results/topic_lists/GLOW_Saraswati1_topics_df.csv", sep=',', encoding = 'utf-8')

In [75]:
recs['g_suggestion'] = True
articles = articles.reindex(sorted(articles.columns), axis=1)
recs = recs.reindex(sorted(recs.columns), axis=1)
pv_df = pv_df.reindex(sorted(pv_df.columns), axis=1)

In [76]:
rec_QIDs= list(recs['QID'].unique())
rec_page_titles = list(recs['page_title'].unique())
rec_page_ids= list(recs['page_id'].unique())
rec_local_wikipedias= list(recs['local_wikipedia'].unique())

rec_Topics= list(recs['Topic'].unique())
rec_article_suggestions= list(recs['article_suggestion'].unique())
rec_local_encoded_titles= list(recs['local_encoded_title'].unique())

print(len(pv_df[pv_df['QID'].isin(rec_QIDs)]))
print(len(pv_df[pv_df['page_id'].isin(rec_page_ids)]))
print(len(pv_df[pv_df['page_title'].isin(rec_page_titles)]))

print(len(pv_df[pv_df['page_title'].isin(rec_local_wikipedias)]))
print(len(pv_df[pv_df['page_title'].isin(rec_Topics)]))
print(len(pv_df[pv_df['page_title'].isin(rec_article_suggestions)]))
print(len(pv_df[pv_df['page_title'].isin(rec_local_encoded_titles)]))

744
4
14
0
91
86
16


In [77]:
recs_select = recs[['g_suggestion','QID', 'database_code', 'given_order', 'suggestion_type',
                         'page_title','page_id','local_wikipedia','Topic', 'article_suggestion', 'local_encoded_title'
                        ]]

recs_select.rename(columns={'given_order'        :'recs_given_order',
                           'suggestion_type'    : 'recs_suggestion_type',
                           'local_wikipedia'    : 'recs_local_wikipedia',
                           'Topic'              :'recs_Topic',
                           'article_suggestion' : 'recs_article_suggestion',
                           'local_encoded_title':'recs_local_encoded_title',
                          }, inplace=True)

In [78]:
m = pv_df.merge(recs_select[['QID','database_code','recs_given_order','recs_suggestion_type', 'g_suggestion']],  how='left', on=['QID', 'database_code']).merge(recs_select[['page_id','database_code','recs_given_order','recs_suggestion_type']], how='left', on=['page_id', 'database_code']).drop(['recs_given_order_y','recs_suggestion_type_y'], axis=1).rename(columns={'given_order':'recs_given_order','recs_given_order_x': 'recs_given_order','recs_suggestion_type_x': 'recs_suggestion_type'})

In [79]:
z = m.merge(recs_select[['page_title','database_code','recs_given_order','recs_suggestion_type', 'g_suggestion']], how='left', on=['page_title', 'database_code'])
z.loc[~z["recs_suggestion_type_y"].isnull(),'recs_suggestion_type_x'] = z["recs_suggestion_type_y"] 
z.loc[~z["recs_given_order_y"].isnull(),'recs_given_order_x'] = z["recs_given_order_y"] 
z = z.drop(['recs_given_order_y','recs_suggestion_type_y'], axis=1).rename(columns={'recs_given_order_x':'recs_given_order','recs_suggestion_type_x': 'recs_suggestion_type'})

z = z.reindex(sorted(z.columns), axis=1)
z["cg_suggestion"] = z[['g_suggestion_x', 'g_suggestion_y']].any(axis='columns')
z = z.drop(['g_suggestion_x','g_suggestion_y'], axis=1).rename(columns={'cg_suggestion':'g_suggestion'})

In [80]:
n = z.merge(recs_select[['recs_Topic','database_code','recs_given_order','recs_suggestion_type', 'g_suggestion']], how='left', left_on=['page_title', 'database_code'], right_on=['recs_Topic', 'database_code'])
n.loc[n["recs_suggestion_type_x"].isnull(),'recs_suggestion_type_x'] = n["recs_suggestion_type_y"] 
n.loc[n["recs_given_order_x"].isnull(),'recs_given_order_x'] = n["recs_given_order_y"] 
n["cg_suggestion"] = n[['g_suggestion_x', 'g_suggestion_y']].any(axis='columns')
n = n.drop(['recs_given_order_y','recs_suggestion_type_y','g_suggestion_x','g_suggestion_y'], axis=1).rename(columns={'recs_given_order_x':'recs_given_order',
                                                                                                                      'recs_suggestion_type_x': 'recs_suggestion_type',
                                                                                                                      'cg_suggestion':'g_suggestion'
                                                                                                                     })


In [81]:
w = n.merge(recs_select[['recs_article_suggestion','database_code','recs_given_order','recs_suggestion_type', 'g_suggestion']], how='left', left_on=['page_title', 'database_code'], right_on=['recs_article_suggestion', 'database_code'])
w.loc[w["recs_suggestion_type_x"].isnull(),'recs_suggestion_type_x'] = w["recs_suggestion_type_y"] 
w.loc[w["recs_given_order_x"].isnull(),'recs_given_order_x'] = w["recs_given_order_y"] 
w["cg_suggestion"] = w[['g_suggestion_x', 'g_suggestion_y']].any(axis='columns')
w = w.drop(['recs_given_order_y','recs_suggestion_type_y','g_suggestion_x','g_suggestion_y'], axis=1).rename(columns={'recs_given_order_x':'recs_given_order',
                                                                                                                      'recs_suggestion_type_x': 'recs_suggestion_type',
                                                                                                                      'cg_suggestion':'g_suggestion'
                                                                                                                     })

In [82]:
z = w.merge(recs_select[['recs_local_encoded_title','database_code','recs_given_order','recs_suggestion_type', 'g_suggestion']], how='left', left_on=['page_title', 'database_code'], right_on=['recs_local_encoded_title', 'database_code'])
z.loc[z["recs_suggestion_type_x"].isnull(),'recs_suggestion_type_x'] = z["recs_suggestion_type_y"] 
z.loc[z["recs_given_order_x"].isnull(),'recs_given_order_x'] = z["recs_given_order_y"] 
z["cg_suggestion"] = z[['g_suggestion_x', 'g_suggestion_y']].any(axis='columns')
z = z.drop(['recs_given_order_y','recs_suggestion_type_y','g_suggestion_x','g_suggestion_y'], axis=1).rename(columns={'recs_given_order_x':'recs_given_order',
                                                                                                                      'recs_suggestion_type_x': 'recs_suggestion_type',
                                                                                                                      'cg_suggestion':'g_suggestion'
                                                                                                                     })

In [83]:
len(z)

1793

In [84]:
recs['suggestion_type'].value_counts()

Editing        8393
Translating    4659
Name: suggestion_type, dtype: int64

In [85]:
articles_labels_df['QID']= articles_labels_df['QID'].str.replace('Q', '')
articles_labels_df['QID'] = articles_labels_df['QID'].astype(np.int64)

In [86]:
final =  articles_labels_df[['QID', 'page_title','predicted_label_1','pl1_score']].merge(z, how='right', on=['page_title', 'QID'])

In [87]:
final = final.drop_duplicates(subset=['page_title', 'database_code', 'page_id', 'filename', 'QID'], keep='first') 

In [88]:
final.to_csv("../../data/raw/articles/2019/query_results/content_quality/per_wiki_full/Indonesia/articles_w_july_incountry_pageviews_recs.csv", sep=',', encoding = 'utf-8')

## India

In [16]:
print('# final submits that were g_suggestions' , len(final_frame_updated_pv_topics_b_a[final_frame_updated_pv_topics_b_a['g_suggestion']==True]))
print('# interim submits that were g_suggestions' , len(interim_frame_updated_pv_topics_suggs[interim_frame_updated_pv_topics_suggs['g_suggestion']==True]))

# final submits that were g_suggestions 855
# interim submits that were g_suggestions 1536
